In [30]:
suppressPackageStartupMessages({
    library(Seurat)
    library(SingleCellExperiment)
    library(qs)
})

In [14]:
PROJ_PATH <- getwd()

In [15]:
source("../Utility/sym_link.r")

In [16]:
View(create_soft_links)

function (input_dir, output_dir, include_dirs = FALSE) 
{
    if (!dir.exists(input_dir)) 
        stop("Input directory does not exist.")
    if (!dir.exists(output_dir)) 
        dir.create(output_dir, recursive = TRUE)
    all_items <- list.files(input_dir, full.names = TRUE, include.dirs = include_dirs)
    for (item in all_items) {
        item_name <- basename(item)
        target_link <- file.path(output_dir, item_name)
        if (!file.exists(target_link)) {
            success <- file.symlink(from = item, to = target_link)
            if (!success) 
                warning(paste("Failed to create symlink for:", 
                  item))
        }
        else {
            message(paste("Link already exists:", target_link))
        }
    }
    message("Symlinks created in: ", output_dir)
}

In [ ]:
create_soft_links(input_dir=file.path(PROJ_PATH, "..", "Step1_Dataset", "in", "SCPCP000001"), output_dir=file.path(PROJ_PATH, "in", "SCPCP000001"), include_dirs = TRUE)

In [18]:
source("../Utility/scpca_data_handling.r")

In [19]:
View(filter_scpca_data)

function (input_dir, output_filepath, cancer_types = c("Glioblastoma"), 
    dtype = "filtered") 
{
    sample_dirs <- list.dirs(input_dir, recursive = TRUE)
    seurat_list <- list()
    metadata_file_path <- file.path(input_dir, "single_cell_metadata.tsv")
    metadata <- read.delim(metadata_file_path, sep = "\t", header = TRUE)
    sample_pick = c()
    counter = 0
    for (i in 1:length(sample_dirs)) {
        sample_id <- basename(sample_dirs[i])
        rds_files <- list.files(path = sample_dirs[i], pattern = paste0("\\_", 
            dtype, ".rds$"), full.names = TRUE)
        for (file in rds_files) {
            test_sample <- TRUE
            sample_id <- strsplit(sub("\\.[[:alnum:]]+$", "", 
                basename(file)), "_")[[1]][1]
            sample_metadata <- metadata[metadata$scpca_library_id == 
                sample_id, ]
            if (any(sample_metadata$diagnosis %in% cancer_types)) {
                counter = counter + 1
                sce_obj <- readRDS(file)
                seurat_obj <- as.Seurat(sce_obj)
                tryCatch({
                  seurat_obj <- as.Seurat(sce_obj)
                }, error = function(e) {
                  seurat_obj <- as.Seurat(sce_obj, data = "counts")
                })
                if (test_sample == FALSE) 
                  next
                seurat_obj$diagnosis <- sample_metadata$diagnosis
                seurat_obj$age <- sample_metadata$age
                seurat_obj$sex <- sample_metadata$sex
                sample_pick <- c(sample_pick, sample_id)
                seurat_list[[sample_id]] <- seurat_obj
            }
        }
    }
    return(seurat_list)
}

In [21]:
metadata_SCPCP000001 <- read.delim(file.path(getwd(),"in/SCPCP000001/single_cell_metadata.tsv"), sep="\t", header=TRUE)
table(metadata_SCPCP000001$diagnosis)


       Anaplastic astrocytoma             Anaplastic glioma 
                            1                             1 
       Diffuse midline glioma                  Glioblastoma 
                            1                            16 
            High-grade glioma                 Non-cancerous 
                            2                             1 
Pleomorphic xanthoastrocytoma 
                            1 

In [22]:
options(warn=-1)
seurat_list_SCPCP000001 <- filter_scpca_data(input_dir="./in/SCPCP000001", 
                                             cancer_types=c("Glioblastoma", 
                                                            "High-grade glioma", 
                                                            "Low-grade glioma",
                                                            "Non-cancerous"), 
                                             dtype="processed")

In [28]:
colnames(seurat_list_SCPCP000001[[name]]@meta.data)

[1] "orig.ident"                     "nCount_originalexp"            
 [3] "nFeature_originalexp"           "barcodes"                      
 [5] "sum"                            "detected"                      
 [7] "subsets_mito_sum"               "subsets_mito_detected"         
 [9] "subsets_mito_percent"           "total"                         
[11] "prob_compromised"               "miQC_pass"                     
[13] "scpca_filter"                   "sizeFactor"                    
[15] "cluster"                        "singler_celltype_ontology"     
[17] "singler_celltype_annotation"    "cellassign_celltype_annotation"
[19] "cellassign_max_prediction"      "diagnosis"                     
[21] "age"                            "sex"

In [31]:
qsave(seurat_list_SCPCP000001, "./out/seurat_list_SCPCP000001.qs")